**### Validation Research Michael Flanderka ###**

Um die Validation implementierung zu verstehen müssen wir erst einmal die Datei "normalization.iypnb" verstehen.

Als ersten Schritt erkläre ich diese:

In [ ]:
import numpy as np
import pandas as pd
import pickle
import gzip

# in this example tanh normalization is used
# fold 0 is used for testing and fold 1 for validation (hyperparamter selection)
norm = 'tanh'
test_fold = 0
val_fold = 1

Hier wird die Normalisierungsstrategie gewählt. Wir nutzen tanh, also eine Begrenzung der Werte zwischen -1 und 1 (grob, nicht eine harte Grenze).

Hierzu wird der Mittelwert entfernt und durch die Standardabweichung geteilt (Standardisierung) und anschließend wird die Hyperbolische Tangens-Funktion (tanh) angewendet.

Die Daten werden in folgender Art und Weise genutzt:

Fold        | Verwendung                    | Variable im Code     
------------|-------------------------------|------------------------
Fold 0      | Testset (nach dem Training)   | X_test, y_test         
Fold 1      | Validierungsset für Tuning    | X_val, y_val           
Folds 2–4   | Trainingsdaten                | X_tr, y_tr             
Folds 1–4   | Trainingsdaten nach Tuning    | X_train, y_train       




2.2 in the paper states:

For data normalization we employed three different types of input normalization: (i) standardizing all inputs to zero mean and unit variance, (ii) standarizing and applying hyperbolic tangent and (iii) standardizing, hyperbolic tangent and standardizing again.

This corresponds to the following code

In [ ]:
def normalize(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:                                                      #If std1 not given: calculate
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:                                                 #If standard deviation is 0 (non informative) throw data away
        feat_filt = std1!=0
    X = X[:,feat_filt]
    X = np.ascontiguousarray(X)                                           #Data array needs to be continuous
    if means1 is None:                                                    #Calc mean and standardize
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1[feat_filt]
    if norm == 'norm':                                                    #Now we start the actual normalization (corresponds to the end of 2.2 in the paper):
        return(X, means1, std1, feat_filt)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        X[:,std2==0]=0
        return(X, means1, std1, means2, std2, feat_filt)

**Parameter:**

X: Eingabematrix (z. B. Merkmale von Proben)

means1, std1: Mittelwert und Standardabweichung der Originaldaten (für erste Normalisierung)

means2, std2: Mittelwert und Standardabweichung nach tanh (für zweite Normalisierung bei tanh_norm)

feat_filt: Filtermaske, die Features mit Standardabweichung = 0 ausschließt

norm: gewählte Normalisierungsstrategie – 'norm', 'tanh', oder 'tanh_norm'

In [ ]:
#contains the data in both feature ordering ways (drug A - drug B - cell line and drug B - drug A - cell line)
#in the first half of the data the features are ordered (drug A - drug B - cell line)
#in the second half of the data the features are ordered (drug B - drug A - cell line)
file = gzip.open('X.p.gz', 'rb')
X = pickle.load(file)
file.close()

In [ ]:
#contains synergy values and fold split (numbers 0-4)
labels = pd.read_csv('labels.csv', index_col=0)
#labels are duplicated for the two different ways of ordering in the data
labels = pd.concat([labels, labels])
#In the end X contains both ordering ways, both labeled the same. This therefor prevents AB and BA from being in different folds and thereby prevents data leakage.

In [ ]:
#remember: test_fold = 0 and val_fold = 1 also see table above!!!
#indices of training data for hyperparameter selection: fold 2, 3, 4
idx_tr = np.where(np.logical_and(labels['fold']!=test_fold, labels['fold']!=val_fold))
#indices of validation data for hyperparameter selection: fold 1
idx_val = np.where(labels['fold']==val_fold)

#indices of training data for model testing: fold 1, 2, 3, 4
idx_train = np.where(labels['fold']!=test_fold)
#indices of test data for model testing: fold 0
idx_test = np.where(labels['fold']==test_fold)

In [ ]:
X_tr    = X[idx_tr]
X_val   = X[idx_val]
X_train = X[idx_train]
X_test  = X[idx_test]

Wir ordnen jetzt die Indixe zu tatsächlichen Daten zu

In [ ]:
y_tr = labels.iloc[idx_tr]['synergy'].values
y_val = labels.iloc[idx_val]['synergy'].values
y_train = labels.iloc[idx_train]['synergy'].values
y_test = labels.iloc[idx_test]['synergy'].values

Und ziehen uns die zugehörigen Zielwerte (Labels) heraus

In [ ]:
if norm == "tanh_norm":
    X_tr, mean, std, mean2, std2, feat_filt = normalize(X_tr, norm=norm)
    X_val, mean, std, mean2, std2, feat_filt = normalize(X_val, mean, std, mean2, std2,
                                                          feat_filt=feat_filt, norm=norm)
else:
    X_tr, mean, std, feat_filt = normalize(X_tr, norm=norm)
    X_val, mean, std, feat_filt = normalize(X_val, mean, std, feat_filt=feat_filt, norm=norm)

X_val wird hier jeweils mit den Parametern aus dem Trainingsdurchlauf normalisiert, um Data Leakage zu vermeiden. Angenommen wir würden auch aus XVal Daten nehmen, um die normalisierung durchzuführen, würde das Modell bereits die Lösung für die Daten kennen und somit besser erscheinen, als es ist.

Wichtig ist hier also, dass die Daten ERST getrennt und DANN die normalisierung berechnet wird. Ansonsten würden bereits statistische Informationen aus der Lösung ins Modell einfließen. Wichtig ist hierbei, dass wir die Parameter aus dem Trainingsdurchlauf weitergeben und NICHT eine erneute Normalisierung für den Validierungsdatensatz machen.

Dieser Durchlauf ist für die **INNERE VALIDIERUNG, also die HYPERPARAMETERAUSWAHL!**

In [ ]:
if norm == "tanh_norm":
    X_train, mean, std, mean2, std2, feat_filt = normalize(X_train, norm=norm)
    X_test, mean, std, mean2, std2, feat_filt = normalize(X_test, mean, std, mean2, std2,
                                                          feat_filt=feat_filt, norm=norm)
else:
    X_train, mean, std, feat_filt = normalize(X_train, norm=norm)
    X_test, mean, std, feat_filt = normalize(X_test, mean, std, feat_filt=feat_filt, norm=norm)

Das hier ist für den Outer Validiation Loop, also die eigentliche Modelloptimierung mit den gefundenen Hyperparametern.

In [ ]:
pickle.dump((X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test),
            open('data_test_fold%d_%s.p'%(test_fold, norm), 'wb'))

Als letztes speichern wir einfach die gesplitteten Daten, um diese exakt so wieder aufrufen zu können.